# Visualizing Airspace Capacity

In the Jupyter notebook 5.1, we gathered basic airspace and traffic data from the triple store, enriched them with additional data and splitted the records into hourly slices. What we have now is a comprehensive dataset of all opened and closed airspaces over time. We are going to display this information as a 5D heatmap on a virtual globe using cesium.js

### Background on Cesium.js

Cesium is a JavaScript project documented at https://cesiumjs.org. Cesiumpy is documented at https://github.com/sinhrks/cesiumpy. In order to fill the globe with airspace definitions, we use the CZML data format, which is a JSON-like data format for Cesium, documented at https://github.com/AnalyticalGraphicsInc/czml-writer/wiki/CZML-Guide.

Basically, CZML is a JSON format serving the Cesium.js with data about objects to be displayed on the globe. One major advantage is that every object attribute can be controlled in a time-based context: a line drawn on the globe could be blue in one moment and red in another. The CZML format is *streaming capable* : this means that the complete document does not need to be present on the client at the moment the globe is displayed.

The Python library cesiumpy works as a wrapper for Cesium.js, which is based on JavaScript. The goal is to get an intuition of the data and to prepare the visualization of the ML results in form of a 4D heatmap. We will show 3D sectors and the time when they were regulated!

### Sample CZML Format

A CZML document is a list of CZML packets:  `[p1, p2, ...]` . Each CZML Packet is in JSON format and describes the graphical properties of an object in the scene. For a minimum working example, we need to describe two objects: 1) a CZML packet describing the object of interest 2) a CZML packet describing the Cesium clock. A MWE could look like this:
```
[{
    "clock":{
      "interval": "2012-04-30T12:00:00Z/14:00:00Z" ,
      "currentTime": "2012-04-30T12:00:00Z",
      "multiplier": 60,
      "range":"LOOP_STOP",
      "step":"SYSTEM_CLOCK_MULTIPLIER"
     }
 }
 {
    "id": "myObject",
    "someProperty": [{
            "interval": "2012-04-30T12:00:00Z/13:00:00Z",
            "number": 5
        },
        {
            "interval": "2012-04-30T13:00:00Z/14:00:00Z",
            "number": 6
        }
    ]
 }
]
```

The ideal entity to display an airspace is *Polygon*, which is described in https://github.com/AnalyticalGraphicsInc/czml-writer/wiki/Polygon. It is important to note that the Coordinates defining the Polygon have to be passed as *PositionList.cartographicDegrees*, which is in the format `[Logitude, Latitude, Height, Longitude, Latitude, Height...]`. The value *Height* is only used if the attribute *perPositionHeight* is set to True. 

### The approach
 1. We construct some CZML creation functions to help create the clock and airspace entities
 2. We pull the data from the triple store and extract a sample of ~5000 from >100.000 airspace openings
 3. We construct a CZML containing 5000 definitions
 4. We pass it to the Cesium widget and display the data!
 
Our CZML airspace function will provide a parameter ranging from -1 to 1 which we will later use to define colors for a heatmap.


# 0. Imports and Essential Functions Definition

In [1]:
import os.path
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import pandas as pd
import json
import cesiumpy
import random
import ast
from geomet import wkt
from pandas.io.json import json_normalize, read_json
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF
from datetime import datetime
from IPython.display import HTML
from IPython.display import clear_output

#Set some parameters for nicer visualizations
pd.set_option('display.expand_frame_repr', False) #do not wrap the printout of Pandas DataFrames
pd.set_option('display.precision', 2)
mp.rcParams['figure.figsize'] = (15, 9)
mp.pyplot.style.use = False


# initialize my connection module which allows to connect oto both datAcron graph databases
from datacron_connector import TripleStoreConnector
ts107 = TripleStoreConnector(0)
ts109 = TripleStoreConnector(1)

#some technical comments
# PREFIX bif: <java:datAcronTester.unipi.gr.sparql_functions.>   <--- only to be used in 109

After the import of all required modules, we load the data that was generated in the previous notebook:

In [2]:
dfob = pd.read_csv('data/7.open_airblocks_hourly.csv')

print('json conversion...')
dfob['actualJSON'] = dfob['actualJSON'].map(ast.literal_eval) #convert string to dict

print('dtype conversion...')
dfob['capacity'] = dfob['capacity'].apply(pd.to_numeric, errors='ignore', downcast='unsigned')
dfob['demand'] = dfob['demand'].apply(pd.to_numeric, errors='ignore', downcast='float')
dfob['ratio'] = dfob['ratio'].apply(pd.to_numeric, errors='ignore', downcast='float')
dfob['lowerlevel'] = dfob['lowerlevel'].apply(pd.to_numeric, errors='ignore', downcast='float')
dfob['upperlevel'] = dfob['upperlevel'].apply(pd.to_numeric, errors='ignore', downcast='float')
print('time conversion...')
dfob['start'] = pd.to_datetime(dfob['start'])
dfob['end'] = pd.to_datetime(dfob['end'])
dfob['duration'] = pd.to_timedelta(dfob['duration'])
dfob.sort_values(by='start', ascending=True, inplace=True)

dfob.sample(3)

json conversion...
dtype conversion...
time conversion...


,Unnamed: 0,config,airspace,sector,block,actualWKT,lowerlevel,upperlevel,start,end,capacity,duration,actualJSON,centerPoint,old_duration,old_capacity,demand,ratio
265117,265117,AirspaceConfiguration_LECPCTA_CONF5D_411,Airspace_LECPCTA_411,Airspace_LECPLNX_411,Airblock_LECPLNX_802LE,"POLYGON ((3.25388888888889 39.8955555555556, 3...",3810.0,6248.4,2016-04-05 13:29:00,2016-04-05 14:28:59,34.99,00:59:59,"{'type': 'Polygon', 'coordinates': [[[3.253888...","[3.05763888888889, 39.77958333333335]",0 days 10:21:00.000000000,35,10,0.29
224182,224182,AirspaceConfiguration_LECMCTAN_CNF9A2_411,Airspace_LECMCTAN_411,Airspace_LECMSAL_411,Airblock_LECMSAL_552LP,"POLYGON ((-8.22833333333333 42.1397222222222, ...",7467.6,10515.6,2016-04-01 09:30:00,2016-04-01 09:55:59,15.59,00:25:59,"{'type': 'Polygon', 'coordinates': [[[-8.22833...","[-4.421805555555555, 41.977361111111094]",0 days 04:25:00.000000000,36,10,0.64
128959,128959,AirspaceConfiguration_LECSCTA_C2A_411,Airspace_LECSCTA_411,Airspace_LECSALMB_411,Airblock_LECSALMB_656LE,"POLYGON ((-1.64638888888889 36.8833333333333, ...",0.0,9296.4,2016-04-12 23:00:00,2016-04-12 23:59:59,24.99,00:59:59,"{'type': 'Polygon', 'coordinates': [[[-1.64638...","[-0.84972222222222, 36.76055555555555]",0 days 01:59:00.000000000,25,0,0.00


## CZML Functions Definitions:

In [3]:
"""
CZML Tools
This cell contains three functions: a CZML document generator, a CZML airspace generator
and a CZML saver function that saves the CZML to a file at a given path.

TODO
 - enrich missing type hints according https://www.python.org/dev/peps/pep-0484/

"""

def create_czml_docinfo(name: str, start_time: str, end_time: str, multiplier: int) -> dict:
    """
    Creates a CZML docinfo JSON-formatted string. 
    Args:
        start_time:  the time at which the clock should start in ISO 8601 format. SPARQL returns ISO 8601
        end_time:    the time at which the clock should end in ISO 8601 format. SPARQL returns ISO 8601
        multiplier: a multiplicator to run the clock at x speed.
    Returns:
        The dictionary that represents a valid CZML object to control the basic clock settings for cesium.
    """
    docinfo = {
        "id": "document",
        "name": name,
        "version":"1.0",
        "clock":{
          "interval": start_time +'/' + end_time ,
          "currentTime": start_time,
          "multiplier": multiplier,
          "range":"LOOP_STOP",
          "step":"SYSTEM_CLOCK_MULTIPLIER"
        }
    }
    
    return docinfo

def rgb(minimum: float, maximum: float, value: float):
    """
    Creates an RGB value using matplotlib colormaps.
    """
    norm = mp.colors.Normalize(vmin=minimum, vmax=maximum)
    cmap = mp.cm.get_cmap('coolwarm')

    rgba = cmap(norm(value))
    r, g, b = rgba[0], rgba[1], rgba[2]
    return r *255, g *255, b *255


def create_czml_airspace(id: str, start_time: str, end_time: str, geojson: dict, heat: float, ll: int, ul: int, dur):
    """
    Creates a CZML packet that represents an airspace. The airspace can be opened and closed by passing
    start and end times. Height will be shown exaggerated by a factor of 10 to facilitate visibility.
    The heat is a floating value between -1 and 1 that controls the color of the airspace from
    -1 being cold to 0 being transparent to 1 being red.
    
    Args:
        id:         An arbitrary airspace ID.
        start_time: Timestring in format YYYY-MM-DDThh:mm:ss  at which the airspace should show on the globe.
        end_time:   Timestring in format YYYY-MM-DDThh:mm:ss  at which the airspace should disappear.
        geojson:    A geoJson dict object containing the 2D coordinates (LonLat) of the airspace.
        heat:       A float value from -1 to 1 representing color intensity (used for heatmaps).
        ll:         Lower level of the airspace in meters above ellipsoid.
        ul:         Uppler level of the airspace in meters above ellipsoid.
    """
 
    # pull coordinates out of geojson:
    coords =[]
    for c in geojson['coordinates'][0]:
        coords.extend(c)
        coords.append(0)
    coords
    
    #do some meter to flight level conversions for the info box:
    llfl = round(ll / 0.3048 / 100)
    ulfl = round(ul / 0.3048 / 100)
    ll = int(ll)
    if ul > 12800:
        ul = 12800
    ul = int(ul)
    
    #control color of airspace with the 'heat' parameter:
    r, g, b = rgb(0, 2, heat)
      
    #deshtml = 'LowerLevel: ' + str(ll)  + 'm (FL' + str(llfl)  + ') <br />Upper Level: ' + str(ul) +'m (FL' + str(ulfl)  +') ' +  str(dur) 
    deshtml = """
        Lower Level:  {s1}m  (FL{s2}) <br />
        Upper Level:  {s3}m  (FL{s4}) <br />
        Opening Time: {s5} <br />
        Duration: {s6} <br />
        Demand/Capacity Ratio (low means low traffic:) {s7:.2f}
        """.format(s1=str(ll), s2=str(llfl), s3=str(ul), s4=str(ulfl), s5=str(start_time), s6=str(dur), s7=heat)
    
    
    airspace = {
        "id": id,
        "description": deshtml,
        "polygon":{
            "positions": {"cartographicDegrees": coords},
            "material": {
                "solidColor":{
                    "color": {"rgba": [r,g,b,75]}
                    }, 
            },
            "fill":True,
            "outline":True,
            "outlineColor":{"rgba": [r,g,b,255]},
            "height": ll * 10,
            "extrudedHeight": (ul *10) - (ll * 10),
            #"show": [{"interval": "2010-01-01T00:00:00Z/" + start_time, "boolean": False },
            #     {"interval": start_time + '/' + end_time, "boolean": True  },
            #     {"interval": end_time + "/2030-03-16T10:00:00Z", "boolean": False  }
            "show": [
                {"interval": start_time + '/' + end_time, "boolean": True  }           
            ]
        },
            
    }
    return airspace


def create_czml(filepath: str, docinfo, *args):
    """
    This function takes a docinfo JSON representing the clock object, and an arbitrary amount of
    other arguments, of which each should represent valid CZML entity definition, and creates
    a CZML file at the specified filepath.
    
    Parameters:
    filepath:  The path of the file to sace the CZML to.
    docinfo:   The clock information entity.
    *args:     An arbitrary number or a list of CZML entities to be inserted into the CZML file.
    """
    myczml = [docinfo]
    myczml.extend(*args)
    with open(filepath, mode='w') as outfile:
        json.dump(myczml, outfile, sort_keys=False, indent=4, ensure_ascii=False)


def updateCesiumHTML(czml_file: str):
    v = cesiumpy.Viewer()
    ds = cesiumpy.CzmlDataSource(czml_file)
    v.dataSources.add(ds)
    myhtml = open('data/cesium.html', mode='w')
    myhtml.write(v.to_html())
    myhtml.close()
    return v
        

## Construct the CZML file

Reminder: we have a comprehensive dataset saved in open_airblocks_hourly. We use this dataset to create the CZML file. This enables us to skip the complete data preparation and jump directly to this chapter (visualization).

In [4]:
#Generate sample data for visualization:
dfviz = dfob.head(10000)

#Create the clock packet:
clock_start = dfviz['start'].min().strftime('%Y-%m-%dT%H:%M:%S')
clock_end   = dfviz['end'].max().strftime('%Y-%m-%dT%H:%M:%S')
multiplier = 500
docinfo = create_czml_docinfo('clock packet', clock_start, clock_end, multiplier)


#Create all the airspace packets:
airspace_list = []
for index, row in dfviz.iterrows():
    c = create_czml_airspace(row['block'] + ' | ' + str(index), 
                             row['start'].strftime('%Y-%m-%dT%H:%M:%S'),
                             row['end'].strftime('%Y-%m-%dT%H:%M:%S'), 
                             row['actualJSON'], row['ratio'], 
                             row['lowerlevel'], row['upperlevel'], 
                             row['duration'] )
    airspace_list.append(c)
    
cz = create_czml('cesiumviz/5_1_airspaceviz.czml', docinfo, airspace_list)

In case of errors or warnings in the next cells, this is due to the Cesium process and may be ignored (simply run the cell again if in doubt)

In [5]:
v = updateCesiumHTML('cesiumviz/5_1_airspaceviz.czml')

/home/joerg/anaconda3/envs/datacron1/lib/python3.6/site-packages/cesiumpy/util/html.py:14: UserWarning: Unable to read specified path, be sure to the output HTML can read the path: 
  warnings.warn(msg.format(sourceUri))


In [6]:
HTML(filename='cesiumviz/cesium.html')